In [43]:
import pandas as pd
import random
import datetime 
from datetime import date

import csv

In [14]:
df = pd.read_csv('../nowon/data/humane_google_ads.csv', on_bad_lines='skip')

In [79]:
df = pd.read_csv('../nowon/data/humane_google_ads.csv')
lista_colunas_remover = ['Codigo da moeda','CTR','CPC medio','% de impr. (1ª posicao)',
                         '% de impr. (parte sup.)','Conv. de visualizacao','Custo / conv.','Taxa de conv.']

lista_colunas_usar = [_ for _ in df.columns if _ not in lista_colunas_remover]
df = df[lista_colunas_usar]

lista_ori_col = [_ for _ in df.columns]
lista_pos_col = ['Date','nome_campanha','cliques','impressoes','investido','conversoes']

dic_col = {ori:pos for (ori,pos) in zip(lista_ori_col,lista_pos_col)}

df.rename(columns=dic_col, inplace=True)

df['conversoes'] = 0
df['interacoes'] = 0

df_filtrado = df.rename(columns=dic_col)

In [95]:
df_filtrado['cliques'] / df_filtrado['cliques'].max() * 10

0      0.748899
1      1.541850
2      1.365639
3      1.497797
4      1.497797
        ...    
92     2.026432
93     4.052863
94     8.678414
95     1.938326
96    10.000000
Name: cliques, Length: 97, dtype: float64

In [63]:
def interar_n(s):
        return int(s)

c_ano, c_mes, c_dia = map(interar_n,df['Date'].min().split("-"))
f_ano_, f_mes, f_dia = map(interar_n,df['Date'].max().split("-"))

start_date = datetime.date(c_ano, c_mes, c_dia+1)
end_date = datetime.date(f_ano_, f_mes, f_dia)

time_between_dates = end_date - start_date
days_between_dates = time_between_dates.days
random_number_of_days = random.randrange(days_between_dates)
data_final = start_date + datetime.timedelta(days=random_number_of_days)

data_atual = data_final
data_antiga = data_final - datetime.timedelta(days=1)

In [74]:
str(data_antiga)

'2022-10-31'

In [75]:
df_atual = df_filtrado[df_filtrado['Date']==str(data_atual)]
df_antigo = df_filtrado[df_filtrado['Date']==str(data_antiga)]

print(data_atual)
display(df_atual)
print(data_antiga)
display(df_antigo)

obj_atual = df_atual[['impressoes','cliques','interacoes','conversoes','investido']].sum()
obj_antigo = df_antigo[['impressoes','cliques','interacoes','conversoes','investido']].sum()


# CTR
ctr_atual = obj_atual.cliques / obj_atual.impressoes
ctr_antigo = obj_antigo.cliques / obj_antigo.impressoes

if ctr_atual > ctr_antigo:
    ctr_diff = '{:.2f}'.format((ctr_atual - ctr_antigo) * 100)
    condicao_ctr = 'Aumentou (⏫)'
elif ctr_antigo > ctr_atual:
    ctr_diff = '{:.2f}'.format((ctr_antigo - ctr_atual) * 100)
    condicao_ctr = 'Diminuiu (⏬)'
else:
    ctr_diff = 0
    condicao_ctr = "Manteve"

# CLIQUES
atual_primeiro_cliques = df_atual.groupby(by='nome_campanha').sum().sort_values(by='cliques',ascending=False).iloc[0]
nome_campanha = atual_primeiro_cliques.name
atual_clique = atual_primeiro_cliques.cliques


antigo_campanha_cliques = df_antigo[df_antigo['nome_campanha'] == nome_campanha].iloc[0]
antigo_clique = antigo_campanha_cliques.cliques

if atual_clique > antigo_clique:
    cliques_diff = '{:.2f}'.format(((atual_clique / antigo_clique)-1)*100)
    condicao_clique = 'teve um aumento (⏫)'
elif antigo_clique > atual_clique:
    cliques_diff = '{:.2f}'.format(((atual_clique / antigo_clique)-1)*100)
    condicao_clique = 'teve uma diminuição (⏬)'
else:
    cliques_diff = 0
    condicao_clique = "manteve a porcentagem"

# CONVERSOES
atual_conv = atual_primeiro_cliques.conversoes  
antigo_conv = antigo_campanha_cliques.conversoes

if atual_conv > antigo_conv:
    conv_diff_por = '{:.2f}'.format(((atual_conv / antigo_conv)-1)*100)
    conv_diff_qtde = '{:.0f}'.format(atual_conv - antigo_conv)
    condicao_conv = 'tiveram um aumento (⏫)'
elif antigo_conv > atual_conv:
    conv_diff_por = '{:.2f}'.format(((atual_conv / antigo_conv)-1)*100)
    conv_diff_qtde = '- {:.0f}'.format(antigo_conv - atual_conv)
    condicao_conv = 'tiveram uma diminuição (⏬)'
else:
    conv_diff_qtde = 0
    conv_diff_por = 0
    condicao_conv = "manteve"

# IMPRESSOES
atual_imp = atual_primeiro_cliques.impressoes  
antigo_imp = antigo_campanha_cliques.impressoes

if atual_imp > antigo_imp:
    imp_diff_por = '{:.2f}'.format(((atual_imp / antigo_imp)-1)*100)
    imp_diff_qtde = '{:.0f}'.format(atual_imp - antigo_imp)
    condicao_imp = 'tiveram um aumento (⏫)'
elif antigo_imp > atual_imp:
    imp_diff_por = '{:.2f}'.format(((atual_imp / antigo_imp)-1)*100)
    imp_diff_qtde = '- {:.0f}'.format(antigo_imp - atual_imp)
    condicao_imp = 'tiveram uma diminuição (⏬)'
else:
    imp_diff_por = 0
    imp_diff_qtde = 0
    condicao_imp = "manteve"

json_resp = {
    'conta_selecionada' : 'Humane',
    'data_atual' : data_atual,
    'data_antiga' : data_antiga,
    'ctr' : {
        'ctr_antigo' : ctr_antigo,
        'ctr_atual' : ctr_atual,
        'ctr_diff' : ctr_diff,
        'condicao_ctr' : condicao_ctr
    },
    'cliques' : {
        'nome_campanha' : nome_campanha,
        'condicao_cliques' : condicao_clique,
        'cliques_diff' : cliques_diff
    },
    'conv' : {
        'nome_campanha' : nome_campanha,
        'condicao_conv' : condicao_conv,
        'conv_diff_por' : conv_diff_por,
        'conv_diff_qtde' : conv_diff_qtde
    },
    'imp' : {
        'condicao_imp' : condicao_imp,
        'imp_diff_por' : imp_diff_por,
        'imp_diff_qtde' : imp_diff_qtde
    }

}
print(json_resp)

2022-11-01


,Date,nome_campanha,cliques,impressoes,investido,conversoes,interacoes
4,2022-11-01,Super Cacau - Search,34,1.357,"50,75",0,0


2022-10-31


,Date,nome_campanha,cliques,impressoes,investido,conversoes,interacoes
3,2022-10-31,Super Cacau - Search,34,1.333,"47,34",0,0


{'conta_selecionada': 'Humane', 'data_atual': datetime.date(2022, 11, 1), 'data_antiga': datetime.date(2022, 10, 31), 'ctr': {'ctr_antigo': 25.506376594148538, 'ctr_atual': 25.055268975681653, 'ctr_diff': '45.11', 'condicao_ctr': 'Diminuiu (⏬)'}, 'cliques': {'nome_campanha': 'Super Cacau - Search', 'condicao_cliques': 'manteve a porcentagem', 'cliques_diff': 0}, 'conv': {'nome_campanha': 'Super Cacau - Search', 'condicao_conv': 'manteve', 'conv_diff_por': 0, 'conv_diff_qtde': 0}, 'imp': {'condicao_imp': 'tiveram um aumento (⏫)', 'imp_diff_por': '1.80', 'imp_diff_qtde': '0'}}


C:\Users\lucas\AppData\Local\Temp/ipykernel_55980/695848679.py:28: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  atual_primeiro_cliques = df_atual.groupby(by='nome_campanha').sum().sort_values(by='cliques',ascending=False).iloc[0]


In [76]:
df[df['Date']=='2022-10-30']

,Date,nome_campanha,cliques,impressoes,investido,conversoes,interacoes
2,2022-10-30,Super Cacau - Search,31,1.419,"52,81",0,0


In [77]:
df[df['Date']=='2022-10-29']

,Date,nome_campanha,cliques,impressoes,investido,conversoes,interacoes
1,2022-10-29,Super Cacau - Search,35,1.179,"102,43",0,0


In [57]:
df['Date'].max()

'2022-12-13'

In [55]:
df

,Date,nome_campanha,cliques,impressoes,investido
0,2022-10-28,Super Cacau - Search,17,1.108,"99,84"
1,2022-10-29,Super Cacau - Search,35,1.179,"102,43"
2,2022-10-30,Super Cacau - Search,31,1.419,"52,81"
3,2022-10-31,Super Cacau - Search,34,1.333,"47,34"
4,2022-11-01,Super Cacau - Search,34,1.357,"50,75"
...,...,...,...,...,...
92,2022-12-11,Performance Max - shop,46,1.126,"43,15"
93,2022-12-12,Super Cacau - Search,92,5.529,"26,43"
94,2022-12-12,Performance Max - shop,197,13.074,"34,82"
95,2022-12-13,Super Cacau - Search,44,3.034,"15,11"


In [54]:
def interar_n(s):
    return int(s)

c_ano, c_mes, c_dia = map(interar_n,df['Date'].min().split("-"))
f_ano_, f_mes, f_dia = map(interar_n,df['Date'].max().split("-"))

start_date = datetime.date(c_ano, c_mes, c_dia+1)
end_date = datetime.date(f_ano_, f_mes, f_dia)

time_between_dates = end_date - start_date
days_between_dates = time_between_dates.days
random_number_of_days = random.randrange(days_between_dates)
random_date = start_date + datetime.timedelta(days=random_number_of_days)

print(random_date - datetime.timedelta(days=1))

2022-11-28


In [30]:
from random import randrange
from datetime import timedelta

def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

random_date('2022/10/20', '2022/12/10')

TypeError: unsupported operand type(s) for -: 'str' and 'str'

# ALTERANDO OS DADOS DO CSV

In [15]:
with open('../nowon/data/humane_google_ads.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        print(', '.join(row))
        print("-----------")

Dia,Campanha,Codigo, da, moeda,Cliques,Impr.,CTR,CPC, medio,Custo,%, de, impr., (1Âª, posicao),%, de, impr., (parte, sup.),Conversoes,Conv., de, visualizacao,Custo, /, conv.,Taxa, de, conv.
-----------
2022-10-28,Super, Cacau, -, Search,BRL,17,1.108,"1,53%","5,87","99,84","79,70%","94,23%","0,00",0,, --,"0,00%"
-----------
2022-10-29,Super, Cacau, -, Search,BRL,35,1.179,"2,97%","2,93","102,43","86,53%","98,63%","0,00",0,, --,"0,00%"
-----------
2022-10-30,Super, Cacau, -, Search,BRL,31,1.419,"2,18%","1,70","52,81","70,77%","99,68%","0,00",0,, --,"0,00%"
-----------
2022-10-31,Super, Cacau, -, Search,BRL,34,1.333,"2,55%","1,39","47,34","52,60%","98,83%","0,00",0,, --,"0,00%"
-----------
2022-11-01,Super, Cacau, -, Search,BRL,34,1.357,"2,51%","1,49","50,75","57,95%","99,00%","0,00",0,, --,"0,00%"
-----------
2022-11-02,Super, Cacau, -, Search,BRL,35,1.648,"2,12%","1,45","50,88","56,52%","98,89%","0,00",0,, --,"0,00%"
-----------
2022-11-03,Super, Cacau, -, Search,BRL,42,1.697,"2,47%","1,

In [29]:
path = '../nowon/data/google_ads.csv'
def criar_dataframe(path):
    try:
        df = pd.read_csv(path)
        return df
    except e:
        print(e)
df = criar_dataframe(path) #.head()

In [8]:
df.head()

,Dia
0,27 de Outubro de 2022 - 13 de Dezembro de 2022


In [75]:
conta_selecionada = random.choice(df['nome_conta'])

In [77]:
df_filtrado = df[df['nome_conta']==conta_selecionada]

In [99]:
date.today().day - 2
data_atual = f'{date.today().year}-{date.today().month}-0{date.today().day - 2}'
data_antiga = f'{date.today().year}-{date.today().month}-0{date.today().day - 3}'

In [109]:
df_atual = df_filtrado[df_filtrado['Date']==data_atual]
obj_atual = df_atual[['impressoes','cliques','interacoes','conversoes','investido']].sum()

ctr_atual = obj_atual.cliques / obj_atual.impressoes


df_antigo = df_filtrado[df_filtrado['Date']==data_antiga]
obj_antigo = df_antigo[['impressoes','cliques','interacoes','conversoes','investido']].sum()

ctr_antigo = obj_antigo.cliques / obj_antigo.impressoes


print(ctr_atual,ctr_antigo)

0.015761104312144555 0.013721198539570703


In [123]:
df_atual.groupby(by='nome_campanha').sum().sort_values(by='cliques',ascending=False).iloc[0].name

C:\Users\lucas\AppData\Local\Temp/ipykernel_54048/1419958202.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_atual.groupby(by='nome_campanha').sum().sort_values(by='cliques',ascending=False).iloc[0].name


'[REDES - HOME-OFFICE] TRÁFEGO (search) - Wi-Force W5 1200 G'

In [127]:
df_antigo[df_antigo['nome_campanha']=='[REDES - HOME-OFFICE] TRÁFEGO (search) - Wi-Force W5 1200 G'].iloc[0]

nome_conta                                    Intelbras - Redes (A7B)
Date                                              2022-12-05 00:00:00
nome_campanha       [REDES - HOME-OFFICE] TRÁFEGO (search) - Wi-Fo...
tipo_rede_ad                                                   SEARCH
impressoes                                                      12155
cliques                                                           960
interacoes                                                        960
conversoes                                                         68
valor_conversoes                                                 68.0
investido                                                      357.17
Name: 73, dtype: object

In [69]:
df.dtypes

nome_conta                  object
Date                datetime64[ns]
nome_campanha               object
tipo_rede_ad                object
impressoes                   int32
cliques                      int32
interacoes                   int32
conversoes                   int32
valor_conversoes           float64
investido                  float64
dtype: object

In [70]:
df.head()

,nome_conta,Date,nome_campanha,tipo_rede_ad,impressoes,cliques,interacoes,conversoes,valor_conversoes,investido
0,Intelbras - Controle de Acesso (A7B),2022-12-06,[CONT-ACESS - EMP-CORP] CONVERSÕES (pmax) - Li...,MIXED,30175,4258,4258,416,416.5,4688.318930
1,Intelbras - Segurança (A7B),2022-12-06,[SEG - TMR] Tráfego (Discovery) - Micro SD,MIXED,14339,119,2176,0,0.0,432.765643
2,Intelbras - Controle de Acesso (A7B),2022-12-06,[CONT-ACESS - INC-ILU] TRÁFEGO (search) - Incê...,SEARCH,12376,663,663,25,25.5,913.325000
3,Intelbras - E-commerce (A7B),2022-12-06,[ES - Off-Grid] Purchase (Search) - LSI 1600 (...,SEARCH,14,3,3,0,0.0,1.040000
4,Intelbras - Segurança (A7B),2022-12-06,[SEG - TMR] Tráfego (Search) - BodyCam,SEARCH,4267,246,246,0,0.0,164.900000


In [13]:
texto_diario = f"""Olá, {nome_usuario}!
Confira como foi o desempenho do Google Ads da {nome_unidade} no dia {data_atual} em comparação com o dia {data_antiga_comparar}. 

{data_atual}, a taxa de cliques (CTR). Aumentou em {número_porcentagem} % ({numero_bruto}) em comparação com o {data_antiga_comparar}.

A campanha {nome_campanha}, {nome_metrica_SIPA_CLICK} aumentou em {numero_porcentagem} %
{data_atual}, os cliques da {nome_campanha} aumentaram em {numero_porcentagem}% ({numero_bruto}) em relação a {data_antiga_comparar}.

{data_atual}, a quantidade de visualização do anúncio diminuiu em {mumero_porcentagem} % ({numero_bruto}) em comparação com {data_antiga_comparar}.

Esse foi seu relatório diário.

Qualquer dúvida, estamos à disposição.

Atenciosamente,

Time NOWON."""

NameError: name 'nome_usuario' is not defined

In [2]:
texto_semanal = """Olá, [nome da pessoa]!
Chegou o seu relatório semanal NOWON!
Confira como foi o desempenho do seu site/ sua página durante a semana de [dia] a [dia] de [mês] de 202… em comparação a [dia] a [dia] de [mês] de 202…

Durante a semana de [dia] a [dia] de [mês] de 202…, a taxa de [nome da métrica] aumentou em [número]% (número) em comparação com a semana de [dia] a [dia] de [mês] de 202… (número).

Durante a semana de [dia] a [dia] de [mês] de 202…, a taxa de [nome da métrica] diminuiu em [número]% (número) em comparação com a semana de [dia] a [dia] de [mês] de 202… (número).

Esse foi o seu relatório semanal de performance da NOWON.
Continue assim, você está fazendo um ótimo trabalho! 

Qualquer dúvida, estamos à disposição.

Atenciosamente,
Time NOWON.
"""

In [ ]:
texto_mensal = """Olá, [nome da pessoa]!
Chegou o seu relatório mensal NOWON!
Confira como foi o desempenho do seu site/ sua página durante o mês [mês] de 202… em comparação ao mês de [mês] de 202…

Parabéns, você conseguiu atingir suas metas para o mês de [mês] e obteve ótimos resultados. Continue assim! 

Infelizmente, no mês de [mês], seus resultados não tiveram um significativo aumento. Mas, não desanime! A NOWON está aqui para ajudá-lo a atingir as suas metas. Let’s keep on the hard work!

Tem alguma dúvida sobre o seu relatório mensal? Não hesite em nos contactar.

Atenciosamente,
Time NOWON."""